In [14]:
# %%file strategy_4.py

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpl_finance as mpf
from scipy import stats
from matplotlib.dates import datestr2num

class TradeStrategySV:
    """
    Trade Strategy 4: Slope and Velocity.
    Overview:
    Use close price to draw a trend, use the derivative to find the slope.
    Use the derivative again to find the velocity for change.
    
    When price raises rapidly, slope > 0, velocity > 0
    When price raises becomes slowly, slope > 0, velocity close 0
    When price peaks, slope = 0, 
    When price falls rapidly, slope < 0, velocity > 0
    When price falls slowly, slope < 0, velocity close to 0 
    When price bottoms, slope = 0,
    
    Therefore, we can use a slope and velocity to find trends and make trades.
    
    Long when price is rising (slope > _buy_slope_threshold > 0) & accelerating (velocity > _buy_velocity_threshold > 0) & Close when price starting to fall (slope < -_close_slope_threshold < 0 and velocity < -_close_velocity_threshold < 0)
    Short when price is falling (slope < _sell_slope_threshold < 0) & accelerating (velocity < _sell_velocity_threshold < 0) & Close when price starting to fall (slope > _close_slope_threshold > 0 and velocity > _close_velocity_threshold > 0)
    
    Assumptions: 
    
    Requirement: 
    1. This strategy should work with any highly liquid security. Preferably Forex or Crypto.
    2. This strategy should work whether trend is up or down, volatile or not. 
    3. The more volatile, the better, as we need to quickly close the position after mean reversion, BEFORE moving_average moved too much. 
    Constants: 
    Params:
    _buy_slope_threshold:       # The slope required to trigger BUY
    _buy_velocity_threshold:    # The velocity required to trigger BUY
    _close_slope_threshold:     # The slope required to trigger CLOSE
    _close_velocity_threshold:  # The slope required to trigger CLOSE
    _sell_slope_threshold:      # The slope required to trigger SELL
    _sell_velocity_threshold:   # The velocity required to trigger SELL
    """
    _buy_slope_threshold = 0.1
    _buy_velocity_threshold = 0.1
    _close_slope_threshold = 0.1
    _close_velocity_threshold = 0.1
    _sell_slope_threshold = -0.1
    _sell_velocity_threshold = -0.1
    
    def __init__(self, leverage:int, margin:float, pip_cost:float, buy_slope_threshold:float = 0.1, buy_velocity_threshold:float = 0.1, close_slope_threshold:float = 0.1, close_velocity_threshold:float = 0.1, sell_slope_threshold:float = -0.1, sell_velocity_threshold:float = -0.1, should_log:bool = False, should_plot:bool = False, axs = None):
        """
        Initialize Strategy with given parameters
        :type leverage: float
        :type margin: float
        :type buy_slope_threshold: float
        :type buy_velocity_threshold: float 
        :type close_slope_threshold: float
        :type close_velocity_threshold: float 
        :type sell_slope_threshold: float
        :type sell_velocity_threshold: float  
        :type should_log: bool
        :type should_plot: bool
        :param leverage: The leverage
        :param margin: The margin interest rate
        :param buy_slope_threshold: # The slope required to trigger BUY
        :param buy_velocity_threshold: # The velocity required to trigger BUY 
        :param close_slope_threshold: # The slope required to trigger CLOSE
        :param close_velocity_threshold: # The slope required to trigger CLOSE 
        :param sell_slope_threshold: # The slope required to trigger SELL
        :param sell_velocity_threshold: # The velocity required to trigger SELL 
        :param should_log Whether or not log the strategy
        :param should_plot Whether or not plot the strategy
        """
        self._leverage = leverage
        self._margin_interest = margin
        self._pip_cost = pip_cost
        self._buy_slope_threshold = buy_slope_threshold
        self._buy_velocity_threshold = buy_velocity_threshold
        self._close_slope_threshold = close_slope_threshold
        self._close_velocity_threshold = close_velocity_threshold
        self._sell_slope_threshold = sell_slope_threshold
        self._sell_velocity_threshold = sell_velocity_threshold
        self._should_log = should_log
        self._should_plot = should_plot
        self._axs = axs
        
    def trade(self, data_df:pd.DataFrame, symbol:str, lots:float, stop_loss:float=0.0):
        """
        Perform Analysis, Decide on Trade, and Execute Trade
        :type data_df: pd.DataFrame
        :type symbol: str
        :type lots: float
        :type stop_loss: float
        :param data_df: The DataFrame for data
        :param symbol: The symbol
        :param lots: The standard lots
        :param stop_loss: number of times of standard deviation
        :return: 
        """
        self._symbol = symbol
        self._lots = lots
            
        """
        Firstly, add `slope` and `velocity` columns to DataFrame
        """
        extra_df = data_df.copy()
        extra_df['profit'] = 0
        extra_df['slope'] = pd.Series(np.gradient(extra_df.pre_close), extra_df.index, name='slope') # Find the slope of yesterday, aka. pre_close
        extra_df['velocity'] = pd.Series(np.gradient(extra_df.slope), extra_df.index, name='velocity')
        
        """
        Secondly, Closing
        """
        extra_df['position'] = "CLOSE"
        
        """
        Thirdly, Buying
        """
        long_mask = (extra_df.slope > self.buy_slope_threshold) & (extra_df.velocity > self.buy_velocity_threshold)
        long_close = (extra_df.slope < -self.close_slope_threshold) & (extra_df.velocity < -self.close_velocity_threshold)
        extra_df.loc[long_mask, 'position'] = "LONG"
        extra_df.loc[long_mask, 'profit'] = (extra_df.loc[long_mask, 'pip_change'] - self._pip_cost * 2) * self._lots * 10 # Assuming 2 transaction pip cost for buying and selling.
        # if stop_loss != 0:
            # TODO: Implement Stop Loss.
        
        """
        Finally, Selling
        """
        short_mask = (extra_df.slope < self.sell_slope_threshold) & (extra_df.velocity < self.sell_velocity_threshold)
        short_close = (extra_df.slope > self.close_slope_threshold) & (extra_df.velocity > self.close_velocity_threshold)
        extra_df.loc[short_mask, 'position'] = "SHORT"
        extra_df.loc[short_mask, 'profit'] = -(extra_df.loc[short_mask, 'pip_change'] - self._pip_cost * 2) * self._lots * 10 # Assuming 2 transaction pip cost for buying and selling.
        # if stop_loss != 0:
            # TODO: Implement Stop Loss.
        
        self._data_df = extra_df
        self._quotes = []
        for _, (d, o, c, h, l) in enumerate(
            zip(
                self._data_df.timestamp, 
                self._data_df.open,
                self._data_df.close,
                self._data_df.high,
                self._data_df.low
            )):
            # Draw Candle Sticks
            d = datestr2num(d)
            # Date, Open, CLose, High, Low tuple 
            val = (d, o, c, h, l)
            # Add val to quotes
            self._quotes.append(val)
        return extra_df

    def plot_trade(self, axs, color_up="green", colordown="red"):
        
        print("Plotting {} results for trade days".format(len(self._data_df.index)))
        mpf.candlestick_ochl(axs, 
                             self._quotes,
                             width=0.6,
                             colorup=color_up,
                             colordown=colordown,
                             alpha=0.75)
        axs.autoscale_view()
        axs.xaxis_date()
        
        # Background Color
        # axs.fill_between(self._data_df.timestamp,
        #                     min(self._data_df.close),
        #                     self._data_df.close,
        #                     color='yellow',
        #                     alpha=.05)
        
        current_index = -1
        current_position = ""
        profit_array = []
        for i, tradeday_row in self._data_df.iterrows():
            if current_position == tradeday_row.position: 
                if self._should_log:
                    # Today is same as yesterday, do nothing
                    print("HOLD {}".format(tradeday_row.timestamp))
            else:
                if current_position == "":
                    # First Day Ever, set initial value
                    current_index = i
                    current_position = tradeday_row.position
                else:
                    # Plot The Trade
                    if current_position == "CLOSE":
                        # If last position was "CLOSED", do nothing
                        if self._should_log:
                            print("CLOSE {}".format(tradeday_row.timestamp))
                    else:
                        if tradeday_row.position == "LONG":
                            profit_array.append(tradeday_row.profit)
                            if self._should_log:
                                print("LONG {} - {}, Profit: ${} ({} pips)".format(self._data_df.timestamp[current_index], tradeday_row.timestamp, round(tradeday_row.profit, 2), round(tradeday_row.pip_change, 1)))
                            axs.fill_between(self._data_df.loc[current_index:i, 'timestamp'],
                                            min(self._data_df.close),
                                            self._data_df.loc[current_index:i, 'close'],
                                            color='green',
                                            alpha=.3)
                            # axs.annotate('{} pips'.format(round(tradeday_row.pip_change, 1)),
                            #              xy=(i, tradeday_row.close+ 0.5),
                            #              arrowprops=dict(facecolor='red'),
                            #              horizontalalignment='left',
                            #              verticalalignment='top',)
                        elif tradeday_row.position == "SHORT":
                            profit_array.append(tradeday_row.profit)
                            if self._should_log:
                                print("SHORT {} - {}, Profit: ${} ({} pips)".format(self._data_df.timestamp[current_index], tradeday_row.timestamp, round(tradeday_row.profit, 2), -round(tradeday_row.pip_change, 1)))
                            axs.fill_between(self._data_df.loc[current_index:i, 'timestamp'],
                                            min(self._data_df.close),
                                            self._data_df.loc[current_index:i, 'close'],
                                            color='red',
                                            alpha=.3)
                            # plt.annotate('{} pips'.format(round(tradeday_row.pip_change, 1)),
                            #              xy=(i, tradeday_row.close + 0.5),
                            #              arrowprops=dict(facecolor='red'),
                            #              horizontalalignment='left',
                            
                            #              verticalalignment='top')
                        
                    # Reset Current Date & Position
                    current_index = i
                    current_position = tradeday_row.position
                
        # Plot Labels
        axs.text(0.95, 
                 0.1, 'Green represents LONG position.',
                 verticalalignment='top', 
                 horizontalalignment='right',
                 transform=axs.transAxes,
                 color='green', 
                 fontsize=16, 
                 bbox={
                     'facecolor': 'green',
                     'alpha': 0.3,
                     'pad': 10
                 })
        axs.text(0.95, 
                 0.18, 'Red represents SHORT position.',
                 verticalalignment='top', 
                 horizontalalignment='right',
                 transform=axs.transAxes,
                 color='red', 
                 fontsize=16, 
                 bbox={
                     'facecolor': 'red',
                     'alpha': 0.3,
                     'pad': 10
                 })
        axs.text(0.05, 
                 0.18, 
                 'Avg. Daily Profit: ${}'.format(round(np.array(profit_array).mean(), 2)),
                 verticalalignment='top', 
                 horizontalalignment='left',
                 transform=axs.transAxes,
                 color='green', 
                 fontsize=16)
        axs.text(0.05, 
                 0.1, 
                 'Std. Daily Profit: ${}'.format(round(np.array(profit_array).std(), 4)),
                 verticalalignment='top', 
                 horizontalalignment='left',
                 transform=axs.transAxes,
                 color='red', 
                 fontsize=16)
        # axs.update_datalim(corners)
        axs.set_xlabel('Trade Dates')
        axs.set_ylabel('Exchange Rates')
        axs.set_title('Forex {}. {}% Return, trading {} lots for {} Days'.format(self._symbol, round(self.trade_profit / (self._lots * 100000 / self._leverage), 2), self._lots, len(self._data_df.index)))
        axs.set_xticks(self._data_df.timestamp.values)
        axs.set_yticks(np.arange(min(self._data_df.close), max(self._data_df.close), 0.0010))
        axs.set_xticklabels(self._data_df.timestamp.values, rotation=90)
        axs.grid(True, alpha=0.1)
        
    @property
    def trade_profit(self):
        return np.sum(self._data_df.profit)
    
    @property
    def buy_slope_threshold(self):
        return self._buy_slope_threshold
    
    @property
    def buy_velocity_threshold(self):
        return self._buy_velocity_threshold
    
    @property
    def close_slope_threshold(self):
        return self._close_slope_threshold
    
    @property
    def close_velocity_threshold(self):
        return self._close_velocity_threshold
    
    @property
    def sell_slope_threshold(self):
        return self._sell_slope_threshold
    
    @property
    def sell_velocity_threshold(self):
        return self._sell_velocity_threshold
    
    @property
    def data_df(self):
        return self._data_df